In [ ]:
import requests
import json
import random
import string
import re
import pandas as pd
import datetime
import time
import sys

In [ ]:
use schema PYTHON_WORKLOADS.azb2c_prd;

In [ ]:
# Get Azure B2C Client ID and Secret (Stored in Snowflake secrets)

from snowflake.snowpark.context import get_active_session
# Snowflake session initialization
session = get_active_session()

tenant_id_query = 'select PYTHON_WORKLOADS.azb2c_prd.get_secret_tenant_id()'
azab2c_prd_tenant_id = session.sql(tenant_id_query).collect()[0][0]

client_id_query = "select PYTHON_WORKLOADS.azb2c_prd.get_secret_client_id()"
azadb2c_prd_client_id = session.sql(client_id_query).collect()[0][0]

client_secret_query = "select PYTHON_WORKLOADS.azb2c_prd.get_secret_client_secret()"
azadb2c_prd_client_secret = session.sql(client_secret_query).collect()[0][0]

In [ ]:
## GET TOKEN

TOKEN_ENDPOINT = "https://login.microsoftonline.com/{0}/oauth2/v2.0/token/".format(azab2c_prd_tenant_id)
SCOPE = "https://graph.microsoft.com/.default"
GRANT_TYPE = "client_credentials"

headers = {
	"content-type": "application/x-www-form-urlencoded"
}

data = {
	"client_id": azadb2c_prd_client_id, # also called appID in some places
	"scope": "https://graph.microsoft.com/.default",
	"client_secret": azadb2c_prd_client_secret,
	"grant_type": "client_credentials"
}

response = requests.post(TOKEN_ENDPOINT, data=data, headers=headers)

# print(response)
bearerToken = json.loads(response.text)['access_token']

In [ ]:
# This cell makes the query to Snowflake and provides the list to be updated
# Print gives list of those users being updated

sql = """with identities as
(
select u.id as b2c_id
,displayname
,givenname
,surname
,id
,streetaddress
,i.value:issuer::varchar as ISSUER
,i.value:issuerAssignedId::varchar as ISSUERASSIGNEDID
,value:signInType::varchar as SIGNINTYPE
from bi_platform_metadata.azure_b2c_partner_prd.users u,table(flatten(input => parse_json(identities))) i
-- from python_workloads.azb2c_cert.users_cert u,table(flatten(input => parse_json(identities))) i
)
select i.b2c_id,um.lead_form_url,i.displayname
from identities i
join bi.partner.user_management um on lower(um.email) = lower(i.issuerAssignedId)
--join bi_platform_metadata.azure_b2c_partner_prd.user_last_signin ls on ls.id = i.b2c_id
where 
    i.signintype = 'emailAddress'
    and (
        replace(i.streetaddress,'http:','https:') <> um.lead_form_url
        or i.streetaddress is null
        )
        """

rows = session.sql(sql).collect()

for row in rows:
    userb2cid = row[0]
    displayname = row[2]
    leadurl = row[1]
    print(userb2cid,displayname,leadurl)

In [ ]:
patch_headers = { 
	"Authorization": bearerToken,
	"Content-Type": "application/json"
}

for row in rows:
    userb2cid = row[0]
    patch_data = '{"streetAddress":"' + row[1] + '"}'
    json_data = json.loads(patch_data)
    # print(patch_data,json_data)
    r = requests.patch('https://graph.microsoft.com/v1.0/users/{0}'.format(userb2cid), json = json_data, headers = patch_headers)
    print(userb2cid,patch_data,r)